In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

# Define start and end dates
start_date = '2023-12-18'
start_date = pd.to_datetime(start_date)
end_date = start_date + pd.Timedelta(days=35)

# Define directories
target = '../Chanae-combined_files'
output_dir = '../Chanae-Graph/2023/winter'

# Define subdirectories for each factor
sixfactor_dir = ['d2m','RH','temperature','rainfall','ws','uvb','durationOfDay']
for dir in sixfactor_dir:
    os.makedirs(os.path.join(output_dir,dir), exist_ok=True)

# Function to create and save plots
def create_plot(df, y_column, y_label, title, file_prefix, sub_dir):
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df[y_column], label=y_label, marker='o')
    plt.xticks(df.index)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel(y_label)
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    output_file = os.path.join(output_dir, sub_dir, f'{file_prefix}_{file.split(".")[0]}.png')
    plt.savefig(output_file, bbox_inches='tight')
    plt.close()
    print(f"Saved graph to {output_file}")

# Process each Excel file
for file in os.listdir(target):
    try:
        df = pd.read_excel(os.path.join(target, file))
        df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
        filtered_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        df = filtered_data
        df.set_index('date', inplace=True)

        # Dew Point
        df_resampled = df.resample('7D').agg({'d2m': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        create_plot(df_resampled, 'd2m', 'Dew Point (°C)', 'Dew Point Over Time (7-day Interval)', 'D2m', sixfactor_dir[0])

        # Relative Humidity
        df_resampled = df.resample('7D').agg({'RH': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        df_resampled['RH'] *= 100  # Convert to percentage
        create_plot(df_resampled, 'RH', 'Relative Humidity (%)', 'Relative Humidity Over Time (7-day Interval)', 'RH', sixfactor_dir[1])

        # Temperature
        df_resampled = df.resample('7D').agg({'minTemp': 'min', 'maxTemp': 'max', 'meanTemp': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        plt.figure(figsize=(10, 6))
        plt.plot(df_resampled.index, df_resampled['minTemp'], label='Min Temp', marker='o')
        plt.plot(df_resampled.index, df_resampled['meanTemp'], label='Mean Temp', marker='o')
        plt.plot(df_resampled.index, df_resampled['maxTemp'], label='Max Temp', marker='o')
        plt.xticks(df_resampled.index)
        plt.title('Temperature Over Time (7-day Interval)')
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True)
        output_file = os.path.join(output_dir, sixfactor_dir[2], f'Temperature_{file.split(".")[0]}.png')
        plt.savefig(output_file, bbox_inches='tight')
        plt.close()
        print(f"Saved graph to {output_file}")

        # Rainfall
        df_resampled = df.resample('7D').sum()
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        df_resampled['tp'] *= 1000  # Convert to millimeters
        create_plot(df_resampled, 'tp', 'Rainfall (mm)', 'Rainfall Over Time (7-day Interval)', 'Rainfall', sixfactor_dir[3])

        # Wind Speed
        df_resampled = df.resample('7D').agg({'WS': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        create_plot(df_resampled, 'WS', 'Wind Speed (m/s)', 'Wind Speed Over Time (7-day Interval)', 'WS', sixfactor_dir[4])

        # UVB
        df['uvb'] = pd.to_numeric(df['uvb'], errors='coerce')
        df_resampled = df.resample('7D').agg({'uvb': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        plt.figure(figsize=(10, 6))
        plt.plot(df_resampled.index, df_resampled['uvb'], label='Radiation', marker='o')
        plt.xticks(df_resampled.index)
        for i, value in enumerate(df_resampled['uvb']):
            plt.text(df_resampled.index[i], value, f'{value:.2f}', ha='left', va='bottom')
        plt.title('Radiation Over Time (7-day Interval)')
        plt.xlabel('Date')
        plt.ylabel('Radiation (J/m^2)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        output_file = os.path.join(output_dir, sixfactor_dir[5], f'Radiation_{file.split(".")[0]}.png')
        plt.savefig(output_file, bbox_inches='tight')
        plt.close()
        print(f"Saved graph to {output_file}")

        # Duration of Day
        df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])
        df_resampled = df.resample('7D').agg({'duration_of_day': 'mean'})
        df_resampled.index = df_resampled.index + pd.DateOffset(days=6)
        df_resampled['duration_hours'] = df_resampled['duration_of_day'].dt.seconds // 3600
        df_resampled['duration_minutes'] = (df_resampled['duration_of_day'].dt.seconds % 3600) // 60
        plt.figure(figsize=(10, 6))
        plt.plot(df_resampled.index, df_resampled['duration_hours'] + df_resampled['duration_minutes'] / 60, label='Duration of Day', marker='o')
        plt.xticks(df_resampled.index)
        for i, (hours, minutes) in enumerate(zip(df_resampled['duration_hours'], df_resampled['duration_minutes'])):
            time_str = f'{int(hours)}h {int(minutes)}m'
            plt.text(df_resampled.index[i], hours + minutes / 60, time_str, ha='left', va='bottom')
        plt.title('Duration of Day Over Time (7-day Interval)')
        plt.xlabel('Date')
        plt.ylabel('Duration (hours)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        output_file = os.path.join(output_dir, sixfactor_dir[6], f'DurationOfDay_{file.split(".")[0]}.png')
        plt.savefig(output_file, bbox_inches='tight')
        plt.close()
        print(f"Saved graph to {output_file}")

    except Exception as e:
        print(f"Error processing file {file}: {e}")
